# Benchmark 
---


This notebook compares the decoder performance between Viterbi Decoder and Neural Decoder [1] on Convolution Codes over AWGN Channel.

Reference:
* [1] Kim, Hyeji, et al. "Communication Algorithms via Deep Learning." ICLR (2018)

### TODOs:
---

* [x] Benchmark Viterbi Decoder 
* [ ] Benchmark Neural Decoder


In [1]:
import numpy as np
import commpy as cp
import multiprocessing as mp

import matplotlib.pyplot as plt
%matplotlib inline

# Dataset

In [2]:
def generate_random_binary_sequence(seq_len, p=0.5):
  """Generate message bits length `seq_len` of a random binary 
  sequence, where each bit picked is a one with probability p.

  Args:
    seq_len: - int - length of message bit
    p - float - probability

  Return:
    seq: - 1D ndarray - represent a message bits
  """
  seq = np.zeros(seq_len)
  for i in range(seq_len):
    seq[i] = 1 if (np.random.random() < p) else 0
  return seq

In [3]:
# For reproducability
np.random.seed(2018)

NUM_SEQS = 100
BLOCK_LEN = 100
DATA_RATE = 1/2
CONSTRAINT_LEN = 3
TRACE_BACK_DEPTH = 5 * CONSTRAINT_LEN

# Generate random message bit sequences
msg_bit_sequences = [None for _ in range(NUM_SEQS)]
for i in range(NUM_SEQS):
    message_bits = generate_random_binary_sequence(BLOCK_LEN)
    msg_bit_sequences[i] = message_bits
    
# Turn on random state again
np.random.seed()

# Define a Trellis structure

In [4]:
#  Generator Matrix (octal representation)
G = np.array([[0o7, 0o5]]) 
M = np.array([CONSTRAINT_LEN - 1])
trellis = cp.channelcoding.Trellis(M, G, feedback=0o7, code_type='rsc')

AttributeError: module 'commpy' has no attribute 'channelcoding'

# Benchmark Decoder

In [ ]:
def awgn_channel(input_signal, snr):
    """Simulate data corruption over AWGN channel"""
    signal = 2.0 * input_signal - 1.0
    noise = snr * np.random.standard_normal(input_signal.shape) 
    return signal + noise

def benchmark(message_bits, snr_dB):
    # Encode message bit
    coded_bits = cp.channelcoding.conv_encode(message_bits, trellis)
    
    # Corrupt message on BAWGN Channel
    coded_bits = awgn_channel(coded_bits, snr_dB)
    
    # make fair comparison between (100, 204) convolutional code and RNN decoder
    # Reference: Author's code
    coded_bits[-2*int(M):] = 0
    
    # Viterbi Decoder on Conv. Code
    decoded_bits = cp.channelcoding.viterbi_decode(
        coded_bits=coded_bits.astype(float), 
        trellis=trellis,
        tb_depth=TRACE_BACK_DEPTH,
        decoding_type='unquantized')
    
    # Number of bit errors (hamming distance)
    hamming_dist = cp.utilities.hamming_dist(
        message_bits.astype(int),
        decoded_bits[:-int(M)])
    
    return hamming_dist

In [ ]:
BERs = []
BLERs = []
SNRs     = np.linspace(-1.5, 2.0, 8)

for snr_dB in SNRs:
    
    snr_dB = np.sqrt(1 / (2*10.**((snr_dB)/10.)))
    with mp.Pool(processes=4) as pool:
        hammming_distances = pool.starmap(
            func=benchmark, 
            iterable=[(msg_bits, snr_dB) for msg_bits in msg_bit_sequences])
    
    # Bit error rate
    ber = sum(hammming_distances) / np.product(np.shape(msg_bit_sequences))
    
    # Block error rate
    bler = np.count_nonzero(hammming_distances) / len(msg_bit_sequences)
    
    BERs.append(ber)
    BLERs.append(bler)
    print('[SNR_dB]={:5.2f} [BLER]={:5.3f} [BER]={:5.7f} '.format(snr_dB, bler, ber))

In [ ]:
plt.figure(figsize=(10, 8))
plt.title('Encoding: Conv. Code || Block Length = 100 || Data rate = 1/2', fontsize=14)
plt.semilogy(SNRs, BERs, '-s')

plt.legend(['Viterbi'], fontsize=16)
plt.xlabel('SNR', fontsize=16)
plt.ylabel('BER', fontsize=16)
plt.grid(True, which='both')
plt.xlim(xmin=SNRs[0], xmax=SNRs[-1])  # this line
plt.ylim(ymin=10e-6, ymax=10e-2)
plt.savefig('plot_100.png')

In [ ]:
# plt.figure(figsize=(10, 8))
# plt.title('Encoding Scheme = Convolution Codes || Block Length = 100 || Data rate = 1/2', fontsize=14)
# plt.semilogy(SNRs, [0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005, 0.00001, 0.0000005], '-s')
# plt.semilogy(SNRs, [0.03, 0.006, 0.003, 0.0004, 0.0002, 0.00006, 0.00007, 0.0000008], '-s')

# plt.legend(['Viterbi', 'Neural Decoder'], fontsize=14)
# plt.xlabel('SNR', fontsize=14)
# plt.ylabel('BER', fontsize=14)
# plt.grid(True, which='both')
# plt.xlim(xmin=0, xmax=SNRs[-1])  # this line
# plt.savefig('plot.png')